In [1]:
import matplotlib.pyplot as plt

import torchvision as tv
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.models as models
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import torch.optim as optim
import os
import time
import loadData8
import itertools
from sklearn.metrics import confusion_matrix

In [2]:
device = torch.device("cuda:0")

In [3]:
batchSize = 4
test_data = loadData8.MyDataset(txt=f'test-8-0.txt', transform=None)
test_loader = DataLoader(dataset=test_data, batch_size=batchSize, shuffle=True, num_workers=2)
print(test_loader)

In [4]:
class Net8(nn.Module):
    def __init__(self):
        super(Net8, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (1, 10))
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (2, 3))
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(20304, 1024)
        self.fc2 = nn.Linear(1024, 500)
        self.fc3 = nn.Linear(500, 54)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
    
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
net8 = Net8()

In [5]:
path = './model_30dB_8个0一组.pkl'
net8.load_state_dict(torch.load(path))
net8 = net8.to(device)

In [6]:
transforms = tv.transforms.Compose([
    # transforms.CenterCrop(64),
    # transforms.CenterCrop(128),
    transforms.ToTensor(),
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [7]:
erroCounter = 0
realLabels = []
predictedLabels = []

In [8]:
correct = 0
total = 0
realLabel = []
predictedLabel = []

In [13]:
 for data in test_loader:
    inputs, labels = data
    print(inputs)
    a = inputs[0:160]
    print("---", a.shape)
    inputs, labels = inputs.cuda(), labels.cuda()
    
    inputs = torch.tensor(inputs, dtype=torch.float32, device = device)

    realLabel.append([int(label) for label in labels])
    outputs = net8(inputs)
    _, predicted = torch.max(outputs.data, dim=1)
    predictedLabel.append([int(label) for label in predicted])
    total += labels.size(0)
    correct += (predicted == labels).sum()
    print("labels: ", labels)
    print("predicted: ", predicted)

tensor([[[[ 0.0449,  0.2918,  0.7135,  ...,  0.5496,  0.2079, -0.0985],
          [-0.9561, -0.9474, -0.7328,  ..., -0.7893, -0.9859, -0.9938]]],


        [[[ 0.5452,  0.2288,  0.1000,  ...,  1.0714,  1.0558,  0.8798],
          [-0.8141, -0.9381, -1.0052,  ...,  0.1696, -0.0122, -0.3649]]],


        [[[ 0.1878,  0.3970,  0.7702,  ...,  0.7528,  0.3166,  0.0657],
          [-0.9557, -0.8312, -0.5925,  ..., -0.6730, -0.9490, -1.0108]]],


        [[[-0.0164,  0.0503,  0.3662,  ...,  0.9018,  0.6105,  0.3144],
          [-0.9714, -0.9536, -0.8741,  ..., -0.3279, -0.6979, -0.8810]]]],
       dtype=torch.float64)
--- torch.Size([4, 1, 2, 1280])
labels:  tensor([ 9, 13, 18, 15], device='cuda:0')
predicted:  tensor([ 9, 53, 18, 15], device='cuda:0')


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [23]:
batchSize = 4
test_data = loadData8.MyDataset(txt=f'test-8-0.txt', transform=None)
test_loader = DataLoader(dataset=test_data, batch_size=batchSize, shuffle=True, num_workers=2)
print(test_loader)

In [24]:
 for data in test_loader:
    inputs, labels = data
    ndarray = inputs.numpy()
    print(ndarray)
    print(ndarray.shape)
#     a = ndarray[0][0][0:160][:]
#     print("---", a.shape)
    inputs, labels = inputs.cuda(), labels.cuda()

[[[[ 0.0448628   0.29176413  0.7135305  ...  0.54955537  0.20790113
    -0.09847145]
   [-0.95614491 -0.94739627 -0.73283993 ... -0.78929549 -0.9859167
    -0.99379567]]]


 [[[ 0.18782226  0.39702025  0.77024612 ...  0.75284793  0.31664693
     0.06566408]
   [-0.95567353 -0.83117127 -0.59251466 ... -0.67301651 -0.94895708
    -1.0107896 ]]]


 [[[ 0.54519952  0.22876496  0.09998038 ...  1.07139942  1.0558073
     0.87980381]
   [-0.81407643 -0.93806789 -1.00519384 ...  0.16957191 -0.01219318
    -0.36490153]]]


 [[[-0.01636786  0.05034531  0.36615405 ...  0.90176236  0.61052213
     0.31439564]
   [-0.97140294 -0.95356692 -0.87412481 ... -0.32788576 -0.69786363
    -0.88100438]]]]
(4, 1, 2, 1280)
